<div class="header">
  <h1 style="float: left;">Visualizing Temporal Networks</h1>
  <img width=300px src="https://www.uni-koblenz-landau.de/de/koblenz/logo.png" style="position: relative;"/>
</div>

In [ ]:
################
# Dependencies #
################
# Note: Never import anything directly always import modules or packages, not single functions or classes,
# We should also check and clean up dependencies at some point.
import fileupload
import IPython.display as ipydisplay
import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import plotly.offline

import vtna.data_import
import vtna.graph
import vtna.layout
import vtna.utility

import main

In [ ]:
#Install extentions:
#https://github.com/peteut/ipython-file-upload
#https://github.com/ipython-contrib/jupyter_contrib_nbextensions
plotly.offline.init_notebook_mode(connected=True)

help_input_graph = """
<input onclick="show_help_graph()" type="image" src="images/help.png" style='max-width: 20px;' alt="Submit Button" />
"""
help_input_metadata = """
<input onclick="show_help_metadata()" type="image" src="images/help.png" style='max-width: 20px;' alt="Submit Button" />
"""
helpscript = """
    <script> 
        function show_help_graph(){
            alert("Graph: Please upload tab-separated values without header, see this example:\\n\\ntimestamp\\t edge 1\\t edge2\\n6123720\\t 12\\t\\t 5\\n6123740\\t 52\\t\\t 2\\n...");
        }
        function show_help_metadata(){
alert(\"Metadata: Please upload tab-separated values without header, see this example:\\n\\nnode\\t attribute\\t attribute\\t\\t...\\n12\\t\\tgreen\\t big\\t\\t\\t...\\n5\\t\\tred\\t\\t small\\t\\t...\\n...\");        }
    </script>"""
display(ipydisplay.HTML(helpscript))

In [ ]:
##################
# CSS/JS Imports #
##################

# Queries
with open('css/queries.css', mode='rt') as f:
    queries_css = f'<style>{f.read()}</style>'
with open('js/queries.js', mode='rt') as f:
    queries_js = f'<script>{f.read()}</script>'
    
ipydisplay.display(ipydisplay.HTML(queries_css))
ipydisplay.display(ipydisplay.HTML(queries_js))

# Metadata sorting drag and drop
with open('css/dragndrop.css', mode='rt') as f:
    dragndrop_css = f'<style>{f.read()}</style>'
# TODO: Use minimal version of js
with open('js/html.sortable.js', mode='rt') as f:
    dragndrop_js = f'<script>{f.read()}</script>'
with open('js/dragndrop.js', mode='rt') as f:
    dragndrop_js += f'<script>{f.read()}</script>'

ipydisplay.display(ipydisplay.HTML(dragndrop_css))
ipydisplay.display(ipydisplay.HTML(dragndrop_js))

In [ ]:
###############
# Data Import #
###############
# TODO: Would be great to have notes on where each layout is used.
bordered_box_layout = widgets.Layout(border='solid 1px rgb(210,210,210)', margin="5px 5px 5px 5px")
box_layout = widgets.Layout(margin="10px 0px 10px 0px")
simbox_layout = widgets.Layout(border='solid 1px rgb(210,210,210)',margin="5px 5px 5px 5px", 
                               width="100%")
upload_layout = widgets.Layout(width="100%")



# Toggle button: Switch between Computer and Network upload
upload_type_toggle_button = widgets.ToggleButtons(
    options=['Computer', 'Network'],
    description='Source:',
    disabled=False,
    button_style='info',
    tooltips=['Upload file from local folder', 'Upload from network path'],
    layout=upload_layout
)
# Text input for Graph file path
graph_data_text = widgets.Text(
    value='',
    description='Graph File:',
    disabled=True,
    layout=upload_layout
)
# Text input for Metadata file path
metadata_text = widgets.Text(
    value='',
    description='Metadata File:',
    disabled=True, 
    layout=upload_layout
)

# Local upload for graph data
local_graph_file_upload = fileupload.FileUploadWidget(    
    label="Upload",
)
# Local upload for metadata
local_metadata_file_upload = fileupload.FileUploadWidget(    
    label="Upload",
)
# Network upload for graph data
network_graph_upload_button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='info',
    tooltip='Upload graph file',
    icon='upload'
)
network_graph_upload_button.layout.display = 'none'
# Network upload for metadata
network_metadata_upload_button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='info', 
    tooltip='Upload metadata',
    icon='upload'
)
network_metadata_upload_button.layout.display = 'none'
# Summary/Error output for graph data upload
graph_data_output = widgets.Output()
with graph_data_output:
    ipydisplay.display(ipydisplay.HTML(help_input_graph))
# Summary/Error output for metadata upload
metadata_output = widgets.Output()
with metadata_output:
    ipydisplay.display(ipydisplay.HTML(help_input_metadata))
# Run button applies changes to metadata and open display view
run_button = widgets.Button(
    description='Display Graph',
    disabled=True,
    button_style='success',
    tooltip='',
    icon='start'
)
# Automatically uploads predefined network datasets and displays graph
autostart_button = widgets.Button(
    description='Autostart',
    disabled=False,
    button_style='warning',
    tooltip='',
    icon='start'
)
# Import menu button is shown in any not Import view and switches back to Import view.
import_menu_button = widgets.Button(
    description='Import',
    disabled=False,
    button_style='warning',
    tooltip='Import',
    icon='start'
)
w_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Loading:',
    bar_style='info',
    orientation='horizontal'
)
w_progress.layout.visibility ='hidden'

# Box for toggle button
upload_type_hbox = widgets.HBox([upload_type_toggle_button], layout=box_layout)
# Box for graph data import
graph_data_upload_hbox = widgets.HBox([graph_data_text, local_graph_file_upload, network_graph_upload_button])
graph_data_configuration_vbox = widgets.VBox([])
w_network_settings = widgets.HBox([graph_data_output, graph_data_configuration_vbox])
import_graph_data_vbox = widgets.VBox([graph_data_upload_hbox, w_network_settings], layout=box_layout)
# Box for metadata import
metadata_upload_hbox = widgets.HBox([metadata_text, local_metadata_file_upload, network_metadata_upload_button])
metadata_configuration_left_vbox = widgets.VBox([], layout=widgets.Layout(margin='0px 10px 0px 0px'))
metadata_configuration_hbox = widgets.HBox([metadata_configuration_left_vbox, metadata_output])
import_metadata_vbox = widgets.VBox([metadata_upload_hbox, metadata_configuration_hbox], layout=box_layout)
# Box for all import functionality
w_toolbar = widgets.HBox([run_button, autostart_button, w_progress])
full_import_vbox = widgets.VBox([upload_type_hbox, import_graph_data_vbox, import_metadata_vbox, w_toolbar])

# Create manager for import functionality
upload_manager = main.UIDataUploadManager(
                     run_button=run_button,
                     local_graph_file_upload=local_graph_file_upload,
                     network_graph_upload_button=network_graph_upload_button,
                     graph_data_text=graph_data_text,
                     graph_data_output=graph_data_output,
                     local_metadata_file_upload=local_metadata_file_upload,
                     network_metadata_upload_button=network_metadata_upload_button,
                     metadata_text=metadata_text,
                     metadata_output=metadata_output,
                     metadata_configuration_vbox=metadata_configuration_left_vbox,
                     column_configuration_layout=box_layout,
                     graph_data_configuration_vbox=graph_data_configuration_vbox
                )
# Attach event handlers to each interactable import widget
upload_type_toggle_button.observe(upload_manager.build_on_toggle_upload_type(), 'value')
local_graph_file_upload.observe(upload_manager.build_handle_local_upload_graph_data(), names='data')
local_metadata_file_upload.observe(upload_manager.build_handle_local_upload_metadata(), names='data')
network_graph_upload_button.on_click(upload_manager.build_handle_network_upload_graph_data())
network_metadata_upload_button.on_click(upload_manager.build_handle_network_upload_metadata())

In [ ]:
######################
# Attribute Ordering #
######################
# Initialize dict that is used by drag n drop sorting/ordering
order_dict = dict()
# Initialize dict that indicates attributes being enabled/disabled
order_enabled = dict()

In [ ]:
################
# Queries Menu #
################
filter_box_layout = widgets.Layout(overflow_y='scroll',
                                   border='solid 1px rgb(210,210,210)',
                                   height='auto',
                                   display='block')

# Global functions to fill by init_queries_manager, used by Javascript engine
addQueryClause = None
deleteQueryClause = None
deleteQuery = None
paintQuery = None
switchQuery = None

def init_queries_manager(metadata, queries_vbox):
    global addQueryClause
    global deleteQueryClause
    global deleteQuery
    global paintQuery
    global switchQuery
    queries_manager = main.UIAttributeQueriesManager(metadata, queries_vbox, filter_box_layout, 
                                                     'html/query.fragment.mustache')
    # Fill functions for Javascript engine
    addQueryClause = queries_manager.build_add_query_clause()
    deleteQueryClause = queries_manager.build_delete_query_clause()
    deleteQuery = queries_manager.build_delete_query()
    paintQuery = queries_manager.build_paint_query()
    switchQuery = queries_manager.build_switch_query()
    
    return queries_manager

In [ ]:
#################
# Graph Display #
#################
# Contains:
# - Display of graph
# - Layout dropdown selection
# - Queries configuration

# Note: Currently I just use time step, instead of time stamp, because it was easier to implement.
# Later we would replace it with timestamps, but this requires to map timestamp -> time step for proper
# functioning.
time_slider = widgets.IntSlider(
    description='Timeline: ',
    min=0,
    step=1,
    continuous_update=False,
)

display_vbox = widgets.VBox()
layout_select_vbox = widgets.VBox()
queries_menu_vbox = widgets.VBox()

# Create Display manager
display_manager = main.UIGraphDisplayManager(time_slider=time_slider, 
                                             display_vbox=display_vbox, 
                                             layout_vbox=layout_select_vbox)

In [ ]:
###########################################
# Menu: Toggle between Import and Display #
###########################################
# Toggle between import tool and animation tool?
simulation_box = None



# Note: We should not build and initialize the simulation box before we have the data.
#     Because without the data a number of functions cannot be properly initiliazed.
#     The global simulation_box is a workaround, it should be replaced by some manager class.
#     It is possible that we should rename the display manager to graph manager,
#     and create a display manager responsible for the simulation box.
def on_run(b):
    # on_run will initialize the simulation_box, which contains the graph display.
    # it will initialize the graph object using the display_manager
    global simulation_box
    global display_manager
    # Apply ordering to metadata
    upload_manager.set_attribute_order(order_dict, order_enabled)
    # Load imported imported graph data and metadata into the temporal graph
    try:
        display_manager.init_temporal_graph(
            edge_list=upload_manager.get_edge_list(),
            metadata=upload_manager.get_metadata(),
            granularity=upload_manager.get_granularity()
        )
    except vtna.graph.MissingNodesInMetadataError:
        upload_manager.display_metadata_upload_error("Invalid metadata (not all nodes are described)")
        return
        
    # Init queries manager and build queries menu
    if upload_manager.get_metadata() is not None:
        queries_manager = init_queries_manager(upload_manager.get_metadata(), queries_menu_vbox)
        queries_manager.register_graph_display_manager(display_manager)
    
    simulation_box = widgets.VBox([import_menu_button,  display_vbox, layout_select_vbox, queries_menu_vbox], 
                                  layout=simbox_layout)
    ipydisplay.display(simulation_box)
    
    display_manager.display_graph()
    
    w_progress.layout.visibility ='visible'
    full_import_vbox.layout.display = 'none'
    simulation_box.layout.display = 'block'
    
def on_autorun(b):
    graph_data_text.value = "http://www.sociopatterns.org/wp-content/uploads/2015/09/primaryschool.csv.gz"
    metadata_text.value = "http://www.sociopatterns.org/wp-content/uploads/2015/09/metadata_primaryschool.txt"
    upload_manager.build_handle_network_upload_graph_data()(None)
    upload_manager.build_handle_network_upload_metadata()(None)
    on_run(None)
    
def on_import(b):
    global simulation_box
    simulation_box.layout.display = 'none'
    w_progress.value=0
    w_progress.layout.visibility ='hidden'
    full_import_vbox.layout.display = 'block'
        
run_button.on_click(on_run)
autostart_button.on_click(on_autorun)
import_menu_button.on_click(on_import)

# Show only Import Box
ipydisplay.display(full_import_vbox)

In [ ]:
display(ipydisplay.HTML('''
<script>
    code_show=true; 
    function code_toggle() {
         if (code_show){
             $('div.input').hide();
         } else {
             $('div.input').show();
         }
         code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
'''))